# Slew, Track and Image taking with ComCam #

This notebook is used for the level 3 integration tests from test plan LVV-P81 (https://jira.lsstcorp.org/secure/Tests.jspa#/testPlan/LVV-P81) as part of test cylce LVV-C176 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCycle/LVV-C176). The following tests are currently run as part of this notebook:

 - LVV-T2290 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2290)
 
Execution steps are separated by horizontal lines. Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA.

In [ ]:
from lsst.ts import utils

# Extract your name from the Jupyter Hub
__executed_by__ = os.environ["JUPYTERHUB_USER"]  

# Extract execution date
__executed_on__ = utils.astropy_time_from_tai_unix(utils.current_tai())
__executed_on__.format = "isot"

# This is used later to define where Butler stores the images
summit = os.environ["LSST_DDS_PARTITION_PREFIX"] == "summit"

print(f"\nExecuted by {__executed_by__} on {__executed_on__}."
      f"\n  At the summit? {summit}")

***
Run the setup.ipnyb notebook to bring all components up and in their enabled position. Check Chronograph.

***

Bring ComCom online and tranistion it to EnabledState. Check Chronograph.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
import os
import sys
import asyncio
import logging

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
comcam = ComCam(domain=domain, log=log)

In [ ]:
comcam.set_rem_loglevel(40)

In [ ]:
await comcam.start_task

In [ ]:
await comcam.enable()

---

Find four targets separated by 5º in azimuth and elevation in a square pattern around az = 120º and el = 60º and rotator angle at PhysicalSky and 1.8º.

At this position, the rotator stays within a couple of degrees of its initial position. This is because the CCW is not running (MTmount in simulation mode).

target_1 -> az = 117.5$^o$, el = 57.5$^o$  
target_2 -> az = 122.5$^o$, el =57.5$^o$  
target_3 -> az = 122.5$^o$, el=62.5$^o$  
target_4 -> az = 117.5$^o$, el = 62.5$^o$  

In [ ]:
target_1 = mtcs.radec_from_azel(az=117.5, el=57.5)
target_2 = mtcs.radec_from_azel(az=122.5, el=57.5)
target_3 = mtcs.radec_from_azel(az=122.5, el=62.5)
target_4 = mtcs.radec_from_azel(az=117.5, el=62.5)

print(f"Target 1: {target_1}\n"
      f"Target 2: {target_2}\n"
      f"Target 3: {target_3}\n"
      f"Target 4: {target_4}\n")

***
Slew to target 1:

In [ ]:
await mtcs.slew_icrs(ra=target_1.ra, dec=target_1.dec, rot_type=RotType.PhysicalSky, rot=1.9)

***
Once on target_1 and tracking, take an image with ComCam

In [ ]:
exp1 = await comcam.take_object(15)
print(f"Target 1 exposure: {exp1}")

---
Slew to target_2:

In [ ]:
await mtcs.slew_icrs(ra=target_2.ra, dec=target_2.dec, rot_type=RotType.PhysicalSky, rot=1.9)

***
Once on target_2 and tracking, take an image with ComCam

In [ ]:
exp2 = await comcam.take_object(15)
print(f"Target 1 exposure: {exp2}")

---
Slew to target_3

In [ ]:
await mtcs.slew_icrs(ra=target_3.ra, dec=target_3.dec, rot_type=RotType.PhysicalSky, rot=1.9)

***
Once on target_3 and tracking, take an image with ComCam

In [ ]:
exp3 = await comcam.take_object(15)
print(f"Target 1 exposure: {exp3}")

---
Slew to target 4

In [ ]:
await mtcs.slew_icrs(ra=target_4.ra, dec=target_4.dec, rot_type=RotType.PhysicalSky, rot=1.9)

---
Once on target_4 and tracking, take an image with ComCam

In [ ]:
exp4 = await comcam.take_object(15)
print(f"Target 1 exposure: {exp4}")

---
Stop tracking to prevent hitting the Rotator soft limit.

In [ ]:
await mtcs.stop_tracking()

***
Use ComCam recent images CCS to ensure that the images were taken (http://ccs.lsst.org/RecentImages/comcam.html). 

***
Query the butler to verify that the images are there and check the metadata. This step must be verified using a separate noteboook. 

***
Wrap Up and Shut Down

This cell is not currently included as part of the test execution, but included here as needed to shutdown the systems

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
await mtcs.standby()

In [ ]:
await comcam.standby()